# SQL

**Learning Objectives**: Learn how to connect to and query SQL databases and get the results back as Pandas DataFrames.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

/usr/local/lib/python3.4/dist-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


## Introduction

[SQL](http://en.wikipedia.org/wiki/SQL), or *Structured Query Lanauge* is a programming language focused on read, writing and transforming table based data sets, called *Relational Databases*. There are a number of reasons you might want to work with data in a SQL database:

* They offer high performance for a limited set of operations. They are much faster than Pandas for many types of operations, but are more limited.
* They can handle data sets that don't fit into memory. Pandas DataFrames have to fit in the memory of your computer, which limits their size. Data bases are typically stored on disk and don't have that limitation. This opens the door for working with much larger data sets. Most "big data" tools offer some sort of SQL interfaces.
* The data you want/need to work with may already be in a SQL database.


There are numerous free and open source databases that offer SQL interfaces:

* [SQLite](http://www.sqlite.org/)
* [MySQL](http://www.mysql.com/)
* [PostgreSQL](http://www.postgresql.org/)

It is a good idea to start with SQLite as it has good performance characteristics, doesn't require any specialized setup and comes already installed with Python.

There are a number of different Python packages for working with SQL databases:

* [SqlAlchemy](http://www.sqlalchemy.org/): foundation layer that other tools rely on.
* [pandas.io.sql](http://pandas.pydata.org/pandas-docs/stable/io.html#io-sql)
* [db.py](https://github.com/yhat/db.py)
* [Blaze](http://blaze.pydata.org/docs/latest/index.html)

For the purpose of this notebook we will be using the [Chinook](http://chinookdatabase.codeplex.com/) demo <b>database</b>. You should start by downloading the `.zip` file and putting the `.sqlite` file in this directory.

In [2]:
ls *.sqlite

Chinook_Sqlite.sqlite*


## Connecting

The first step in working with an existing SQL database is to connect to it. This involves figuring out the appropriate connection string and passing it to `sqlalchemy.create_engine`. Here is the logic for our SQLite database:

In [3]:
from sqlalchemy import create_engine 

In [4]:
te = create_engine('sqlite:///titanic.db')

Notice, this will use the file `titanic.db` in the current directory, which doesn't yet exist. SqlAlchemy will create it when needed. Here is the SqlAlchemy documentation for [engines and connection strings](http://docs.sqlalchemy.org/en/rel_0_9/core/engines.html).

## `pandas.io.sql`

Pandas has basic capabilties to reading, writing and querying SQL databases. These function are in the `pandas.io.sql` subpackage:

In [5]:
from pandas.io import sql

In [6]:
tdf = sns.load_dataset('titanic')

In [7]:
tdf.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


You can write a table to a SQL database using the `to_sql` method, which takes the name of the table and an engine object:

In [8]:
tdf.to_sql('people', te)

Now that we have a populated database table, we can use Pandas to query the data set. The first thing we might want to do is read the entire table back into Python as a `DataFrame`. To emphasize that this is a new object, let's delete the original `DataFrame` we had.

In [9]:
del tdf

Then use the `read_sql_table` function with the table name and engine object to get the entire table:

In [10]:
new_tdf = sql.read_sql_table('people', te)

In [11]:
new_tdf.head()

,index,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,0,3,male,22,1,0,7.2500,S,Third,man,True,None,Southampton,no,False
1,1,1,1,female,38,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,2,1,3,female,26,0,0,7.9250,S,Third,woman,False,None,Southampton,yes,True
3,3,1,1,female,35,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,4,0,3,male,35,0,0,8.0500,S,Third,man,True,None,Southampton,no,True


Warning: don't do this unless you know exactly how large the table is. Databases can contain tables that are many times larger than the available memory on your system.

The real power of SQL databases comes from being able to query them using the SQL language. This can be done in Pandas using the `read_sql_query` function. To use this function, we need to cover the basic syntax of the SQL language. Here is a [high-level overview](http://www.w3schools.com/sql/default.asp) of the SQL language.

### SELECT

The `SELECT` statement is the most important part of the SQL language and is used to reads rows from a database table or tables. Here is an example that reads all (`*`) rows from the `people` table:

In [12]:
sql.read_sql_query('SELECT * FROM people;', te).head()

,index,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,0,3,male,22,1,0,7.2500,S,Third,man,1,None,Southampton,no,0
1,1,1,1,female,38,1,0,71.2833,C,First,woman,0,C,Cherbourg,yes,0
2,2,1,3,female,26,0,0,7.9250,S,Third,woman,0,None,Southampton,yes,1
3,3,1,1,female,35,1,0,53.1000,S,First,woman,0,C,Southampton,yes,0
4,4,0,3,male,35,0,0,8.0500,S,Third,man,1,None,Southampton,no,1


You can read particular rows by name as follows:

In [13]:
sql.read_sql_query('SELECT survived, sex, age, fare FROM people;', te).head()

,survived,sex,age,fare
0,0,male,22,7.2500
1,1,female,38,71.2833
2,1,female,26,7.9250
3,1,female,35,53.1000
4,0,male,35,8.0500


### LIMIT

You can use the `LIMIT` clause to limit the number of rows that are returned. This is much more efficient that reading all  rows and then using `.head()`.

In [14]:
sql.read_sql_query("""
SELECT survived, sex, age, fare FROM people
    LIMIT 10;
""", te)

,survived,sex,age,fare
0,0,male,22,7.2500
1,1,female,38,71.2833
2,1,female,26,7.9250
3,1,female,35,53.1000
4,0,male,35,8.0500
5,0,male,NaN,8.4583
6,0,male,54,51.8625
7,0,male,2,21.0750
8,1,female,27,11.1333
9,1,female,14,30.0708


### WHERE

You can use the `WHERE` clause to pick a subset of rows that satisfy certain criteria or tests. Notice how the notation for these tests is sligthly different from Python and Pandas.

In [15]:
sql.read_sql_query("""
SELECT survived, sex, age, fare
    FROM people
    WHERE age>20 AND survived=0 and sex="male";
""", te).head()

,survived,sex,age,fare
0,0,male,22,7.2500
1,0,male,35,8.0500
2,0,male,54,51.8625
3,0,male,39,31.2750
4,0,male,35,26.0000


### ORDER BY

You can sort the resulting rows using the `ORDER BY` clause and the name of a column or columns

In [16]:
sql.read_sql_query("""
SELECT survived, sex, age, fare
    FROM people
    WHERE age>20 AND survived=0 and sex="male"
    ORDER BY fare;
""", te).head()

,survived,sex,age,fare
0,0,male,36,0
1,0,male,40,0
2,0,male,49,0
3,0,male,39,0
4,0,male,38,0


Add `DESC` to reverse the order of the sort to descending:

In [17]:
sql.read_sql_query("""
SELECT survived, sex, age, fare
    FROM people
    WHERE age>20 AND survived=0 and sex="male"
    ORDER BY fare, age DESC;
""", te).head(10)

,survived,sex,age,fare
0,0,male,49.0,0.0000
1,0,male,40.0,0.0000
2,0,male,39.0,0.0000
3,0,male,38.0,0.0000
4,0,male,36.0,0.0000
5,0,male,33.0,5.0000
6,0,male,61.0,6.2375
7,0,male,34.5,6.4375
8,0,male,43.0,6.4500
9,0,male,34.0,6.4958


### SQL functions

SQL offers a host of functions that can applied to columns of data:

* `AVG()`
* `COUNT()`
* `MIN()`
* `MAX()`
* `SUM()`

Here we compute the number of males and the average fare they paid:

In [18]:
sql.read_sql_query("""
SELECT COUNT(sex), AVG(fare)
    FROM people
    WHERE sex="male";
""", te)

,COUNT(sex),AVG(fare)
0,577,25.523893


### GROUP BY

The `GROUP BY` clause allows you to perform split-apply-combine to a table to perform a wide range of transformations and aggregations. This works almost identically to the `groupby` method in Pandas. Here we using `GROUP BY` to compute averarge fares by gender:

In [19]:
sql.read_sql_query("""
SELECT sex, AVG(fare)
    FROM people
    GROUP BY sex
""", te)

,sex,AVG(fare)
0,female,44.479818
1,male,25.523893


Here is the same computation on a Pandas `DataFrame`:

In [20]:
new_tdf.groupby('sex')['fare'].mean()

sex
female    44.479818
male      25.523893
Name: fare, dtype: float64